In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from workflow_widgets import WorkflowWidget, WorkerPoolWidget
from droplet_workflow import droplet_wf
from example_workflow import example_wf
import workflow_objects as kale
import ipywidgets as ipw

In [4]:
import traitlets as tr

# Example Workflow

In [7]:
WorkflowWidget(example_wf)

A Jupyter Widget

In [ ]:
example_wf.get_task_by_name('echo_task_2').log_path

In [ ]:
wpw = WorkerPoolWidget()
wpw

In [ ]:
wpw.add_pool("Example Pool", 4)

In [ ]:
pool = wpw.get_pool("Example Pool")
pool.fw_run(example_wf)

---

# Droplet Workflow

In [ ]:
WorkflowWidget(droplet_wf)